In [44]:
#Import the different packages needed
from IPython.display import display
import pandas as pd
#Changes the default feature when displaying
pd.options.display.max_columns = None # Display all columns of a dataframe
pd.options.display.max_rows = 30000
from pprint import pprint
import re
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

In [45]:
#Get our webdriver
chromeOptions = webdriver.ChromeOptions()
#it is way to add options

path = "C:/Users/User/Documents/GitHub/Films_DM_2023/chromedriver_win32" #the path

driver = webdriver.Chrome(executable_path=os.path.join(path, 'chromedriver'))

#AT this point in time, it only open a Google page

C:\Users\User\AppData\Local\Temp\ipykernel_77176\2552225005.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=os.path.join(path, 'chromedriver'))


In [46]:
#Goes to the Allocine website
driver.get("https://www.allocine.fr/")
time.sleep(1) # Let the user actually see something!

In [47]:
#Accept cookies to be able to go further (it is also the reason why we put the cells 3 and 4, indeed, if we directly scrapped from the other pages,)
#we would only have cookies for the firstpage and not the other).
cookies = driver.find_element(By.CLASS_NAME, "jad_cmp_paywall_button-cookies").click()
time.sleep(2)

In [51]:
#webscraping in itself

#1) initialize the different lists needed to store information to later form the Dataframes
films =[]
films_addedinfo=[]
real_info=[]
act1_info=[]
act2_info=[]

#2) create a function webscraping the page (as we will have to do it multiple times => efficiency gains)
def my_function():
    
    #First create the list that will be used to navigate between the different parts (indeed for each page with 15 films, we will then navigate for EACH film to:
    #1) the page with more info on the film (links)
    #2) the page on the director (links_real)
    #3) the page on the first actor cited (links_actor1)
    #4) the page on the second actor cited (links_actor2)
    
    #If we put them before the function, they are not reinitialized each time which is problematic
    links=[]
    links_real=[]
    links_actor1=[]
    links_actor2=[]
    
    #First, separate each film (=> get the "box" about each film in the list)
    #=> we take film by film the 15 movies available on a page
    #We can see from the code of the website that these information are in a list <li>
    
    for each_fiche in driver.find_elements(By.CLASS_NAME,'mdl'):
        
        #titles of the films
        titre=each_fiche.find_element(By.CLASS_NAME, "meta-title-link").text
        
        #release date (or date_sortie in French)
        date_sortie= each_fiche.find_element(By.CLASS_NAME, "date").text
        
        #durée (length)
        duree =each_fiche.find_element(By.CLASS_NAME, "meta-body-info").text.split('/')[1]
        
        #As duree is not included in the span, we have to go higher to get the tag including the length (=> in the div)
        #However, when we take the text in the div, we have the release date/ the length / the genres (with, between them)
        #So we have to select the part of interest (which is done thanks to split ('/')[1])
    
        #Genres
        info =each_fiche.find_element(By.CLASS_NAME, "meta-body-info")
        genres= info.find_elements(By.CLASS_NAME, "xXx")
        
        #Initiate the variables (otherwise, pb with local and global variables)
        genre_1="NA"
        genre_2="NA"
        genre_3="NA"
        
        #genre1
        genre_1 = genres[0].text
        #Genre 2 et 3
        if len(genres)>=2:
            genre_2= genres[1].text
            if len(genres)==3:
                genre_3=genres[2].text
        
        #Directors
        if each_fiche.find_elements(By.CLASS_NAME, "meta-body-direction"):
            direction =each_fiche.find_element(By.CLASS_NAME, "meta-body-direction")
            #This gives the subpart with information on direction
        
            realisateurs= direction.find_elements(By.CSS_SELECTOR, "a")
    
            #Get the link to the first realisator page to get info if he got awards later
            link_real= realisateurs[0].get_attribute('href')
            links_real.append(link_real)
        
    
            #realisateur1
            real_1 = realisateurs[0].text
            #real 2
            if len(realisateurs)>=2:
                real_2= realisateurs[1].text
            else:
                real_2="NA"
        else:
            link_real="NA"
            links_real.append(link_real)
            real_1="NA"
            real_2="NA"
    
        #Actors
        
        #First if actors are listed
        if each_fiche.find_elements(By.CLASS_NAME, "meta-body-actor"):
            #This gives the subpart with info on the actors
            casting =each_fiche.find_element(By.CLASS_NAME, "meta-body-actor")
            #This gives the subpart with info on the actors
            actors= casting.find_elements(By.CSS_SELECTOR, "a")
    
            #Get the link to the actors 1
            link_actor1= actors[0].get_attribute('href')
            links_actor1.append(link_actor1)
    
    
            #actor1
            actor_1 = actors[0].text
            #actor 2
            if len(actors)>=2:
                actor_2= actors[1].text
                #Get the link to the actors 2
                link_actor2= actors[1].get_attribute('href')
                links_actor2.append(link_actor2)
                if len(actors)==3:
                    actor_3=actors[2].text
                else:
                    actor_3="NA"
            else:
                actor_2="NA"
                actor_3="NA"
                link_actor2= "NA"
                links_actor2.append(link_actor2)
                
        else: #So if actors are not listed
            actor_1="NA"
            actor_2="NA"
            actor_3="NA"
            link_actor1= "NA"
            link_actor2= "NA"
            links_actor1.append(link_actor1)
            links_actor2.append(link_actor2)
        
        #Ratings
        #First initiate them in NA if does not exist
        note_spectateur="NA"
        note_presse="NA"
        
        #Look for the ratings
        if each_fiche.find_elements(By.CLASS_NAME, "rating-holder"): #This first if checks if rating-holder exists => indeed, some films have no rating at all
            rating = each_fiche.find_element(By.CLASS_NAME, "rating-holder")
            ratings=rating.find_elements(By.CLASS_NAME,'rating-item-content')
            
            #For the first one
            #We have to test both PRESSE and SPECTATEURS as if presse exists it will be in the first position but if it odes not it will be spectateurs
            rating_type_1=ratings[0].find_element(By.CLASS_NAME,'rating-title').text
            if(rating_type_1== "PRESSE"):
                note_presse= ratings[0].find_element(By.CLASS_NAME, 'stareval-note').text
            elif(rating_type_1== "SPECTATEURS"):
                note_spectateur= ratings[0].find_element(By.CLASS_NAME, 'stareval-note').text 
        
            #For the second one
            if ratings[1].find_element(By.CLASS_NAME,'rating-title').text:
                rating_type_2=ratings[1].find_element(By.CLASS_NAME,'rating-title').text
                if(rating_type_2== "SPECTATEURS"):
                    note_spectateur= ratings[1].find_element(By.CLASS_NAME, 'stareval-note').text 
                elif(rating_type_2== "PRESSE"): #sometimes it is not in the same order
                    note_presse= ratings[1].find_element(By.CLASS_NAME, 'stareval-note').text
    
        #Now we are done with the information we could find on the first page => click on the link for each film and go see more info
        #Problem, if come back to main page, stale elements => work around it by first creating a vector with all the links and visiting at the end without coming back to the main page
        link= each_fiche.find_element(By.CLASS_NAME, "meta-title-link").get_attribute('href')
        links.append(link)
    

        films.append({
                "Titre": titre,
                "date_sortie" : date_sortie,
                "duree": duree,
                "genre1": genre_1,
                "genre2": genre_2,
                "genre3": genre_3,
                "réalisateur_1": real_1,
                "réalisateur_2": real_2,
                "actor_1": actor_1,
                "actor 2": actor_2,
                "actor 3": actor_3,
                "note_presse": note_presse,
                "note spectators": note_spectateur,
                "link":link,
                "link realisator": link_real,
            })
    
    #Get info on second page (more info on films)
    for link in links:
        driver.get(link)
        time.sleep(2)
        infos = driver.find_element(By.CLASS_NAME,'meta-body')
        #Scenaristes
        
        scenar_1="NA" #initialize the value
    
        if infos.find_elements(By.XPATH,'div[3]'):
            scenar = infos.find_element(By.XPATH,'div[3]')
            if scenar.find_element(By.CSS_SELECTOR,"span").text == "Par":
                scenarists = scenar.find_elements(By.CSS_SELECTOR,'a')
                scenar_1 = scenarists[0].text
                
            elif infos.find_elements(By.XPATH,'div[4]'): #To take into account the fact to have a date before
                scenar = infos.find_element(By.XPATH,'div[4]')
                if scenar.find_element(By.CSS_SELECTOR,"span").text == "Par":
                    scenarists = scenar.find_elements(By.CSS_SELECTOR,'a')
                    scenar_1 = scenarists[0].text
                    
    
        #Technical informations
        techni= driver.find_element(By.CLASS_NAME, 'ovw-technical')
        #nationality
        natio_1="NA"
        natio_2="NA"
        
        natios = techni.find_elements(By.CLASS_NAME, 'nationality')
        if len(natios)==1:
            natio_1 = natios[0].text
            natio_2 = "NA"
        elif len(natios) >= 2:
            natio_1 = natios[0].text
            natio_2 = natios[1].text
            
            
        #Distributor
        nom=techni.find_element(By.XPATH,'div[3]/span').text
        if(nom == "Distributeur"):
            if techni.find_elements(By.XPATH,'div[3]/a'):
                distributor = techni.find_element(By.XPATH,'div[3]/a').text
            else: 
                distributor="NA"
            
        #Palmares
    
        #Attention, this does not exist for all the films!!!
        
        #Initialize
        
        budget="NA"
        boxeoffice="NA"
        
        #If it exists
        if(techni.find_element(By.XPATH,'div[4]/span').text == "Récompense" or techni.find_element(By.XPATH,'div[4]/span').text == "Récompenses" ):
            
            recompense = techni.find_element(By.XPATH,'div[4]/a').text
            recompense=str(re.sub("[^0-9^]"," ",recompense)) #not to have the letters (but still a blank because sometimes awards and sometimes nomination)
            
            #As if there is a recompense, there is one row more in the table with infos, it changes our div => put for the ones with recompenses here
            budget=0
        
            #Box office
            exist=techni.find_element(By.XPATH,'div[11]/span').text
            if(exist == "Box Office France"):
                boxeoffice=techni.find_element(By.XPATH,'div[11]/a').text
            elif (exist == "Budget"): #As there is not always a raw boxoffice
                budget=techni.find_element(By.XPATH,'div[11]/span[2]').text
            
            #Budget
            exist2=techni.find_element(By.XPATH,'div[12]/span').text
            if(exist2 == "Budget"):
                budget=techni.find_element(By.XPATH,'div[12]/span[2]').text
            elif (exist2 == "Langues"): #As there is not always a raw boxoffice and this repercutes
                language=techni.find_element(By.XPATH,'div[12]/span[2]').text
            
            #Language
            exist6=techni.find_element(By.XPATH,'div[13]/span').text
            if(exist6 == "Langues"):
                language=techni.find_element(By.XPATH,'div[13]/span[2]').text
            exist7=techni.find_element(By.XPATH,'div[14]/span').text
            if(exist7 == "Langues"):
                language=techni.find_element(By.XPATH,'div[14]/span[2]').text
        
        
        else:  #If the film has no recompense field 
            recompense= "0"
    
            #Box office
            exist3=techni.find_element(By.XPATH,'div[10]/span').text
            if(exist3 == "Box Office France"):
                boxeoffice=techni.find_element(By.XPATH,'div[10]/a').text
            elif (exist3 == "Budget"):
                budget=techni.find_element(By.XPATH,'div[10]/span[2]').text
            
            #Budget
            exist4=techni.find_element(By.XPATH,'div[11]/span').text
            if(exist4 == "Budget"):
                budget=techni.find_element(By.XPATH,'div[11]/span[2]').text
            elif (exist4 == "Langues"):
                language=techni.find_element(By.XPATH,'div[11]/span[2]').text
            
            #Language
            exist5=techni.find_element(By.XPATH,'div[12]/span').text
            if(exist5 == "Langues"):
                language=techni.find_element(By.XPATH,'div[12]/span[2]').text
  

        films_addedinfo.append({
                "Scenar": scenar_1,
                "nationality 1": natio_1,
                "nationality 2": natio_2,
                "distributor": distributor,
                "nominations": recompense,
                "budget": budget,
                "language": language,
                "box-office":boxeoffice,
            })
    
    
    #Get more info on the realisator

    for real in links_real:
        if real != "NA":
            driver.get(real)
            time.sleep(2)
            if driver.find_elements(By.CLASS_NAME,'stats-numbers-row'): #Check if there is a stats-number-row, indeed, some realisators do not have any information on them
                real_infos = driver.find_element(By.CLASS_NAME,'stats-numbers-row')
                if real_infos.find_elements(By.CSS_SELECTOR,'a'):
                    award_real = real_infos.find_element(By.XPATH,"//a/div[@class='stats-number']").text
                else: 
                    award_real=0
            else:
                award_real="NA"
        else:
            award_real="NA"
            
        real_info.append({"real awards":award_real,})
    
    
    #Get more info on the actors 1
    for actor1 in links_actor1:
        if actor1 != "NA":
            driver.get(actor1)
            time.sleep(1.5)
            if driver.find_elements(By.CLASS_NAME,'stats-numbers-row'):
                act1_infos = driver.find_element(By.CLASS_NAME,'stats-numbers-row')
                if act1_infos.find_elements(By.CSS_SELECTOR,'a'):
                    award_act1 = act1_infos.find_element(By.XPATH,"//a/div[@class='stats-number']").text
                else: 
                    award_act1=0
            else: 
                award_act1="NA"
        else:
            award_act1=0
            
        act1_info.append({"first actor awards":award_act1,})

    #Get more info on the actors 2
    for actor2 in links_actor2:
        
        if actor2 != "NA":
            driver.get(actor2)
            time.sleep(1.5)
            if driver.find_elements(By.CLASS_NAME,'stats-numbers-row'):
                act2_infos = driver.find_element(By.CLASS_NAME,'stats-numbers-row')
                if act2_infos.find_elements(By.CSS_SELECTOR,'a'):
                    award_act2 = act2_infos.find_element(By.XPATH,"//a/div[@class='stats-number']").text
                else: 
                    award_act2=0
            else:
                award_act2="NA"
        else: award_act2=0
        act2_info.append({"Second actor awards":award_act2,})

#Get it for different pages


for page in range(1,2589,50): #By step of 50 pages until 2589 as after they have no known date
    raw_url = "https://www.allocine.fr/films/alphabetique/?page={}"
    driver.get(raw_url.format(str(page)))
    time.sleep(1) # Let the user actually see something!
    time.sleep(2)
    my_function()
    
    
#dataframe
films_data = pd.DataFrame(films)
films_added= pd.DataFrame(films_addedinfo)
real_info=pd.DataFrame(real_info)
act1_info=pd.DataFrame(act1_info)
act2_info=pd.DataFrame(act2_info)

#Concatenate the different df to have one final one
Movies = pd.concat([films_data, films_added,real_info,act1_info,act2_info], axis=1)
Movies

#Pass it in CSV to have a file to use later (indeed, selenium is moving so we want to have one for once and all)

Movies.to_csv('movies_data_morecomprehensive.csv', encoding='utf-8-sig',sep=";", index=False)

In [52]:
Movies

,Titre,date_sortie,duree,genre1,genre2,genre3,réalisateur_1,réalisateur_2,actor_1,actor 2,actor 3,note_presse,note spectators,link,link realisator,Scenar,nationality 1,nationality 2,distributor,nominations,budget,language,box-office,real awards,first actor awards,Second actor awards
0,« Peu m'importe si l'Histoire nous considère c...,20 février 2019,2h 19min,Comédie,Drame,NA,Radu Jude,NA,Ioana Iacob,Alexandru Dabija,Alexandru Bogdan,"3,8","3,3",https://www.allocine.fr/film/fichefilm_gen_cfi...,https://www.allocine.fr/personne/fichepersonne...,Radu Jude,Roumanie,France,Météore Films,0,-,Roumain,901 entrées,6,0,0
1,# Pire soirée,2 août 2017,1h 41min,Comédie,NA,NA,Lucia Aniello,NA,Scarlett Johansson,Kate McKinnon,Jillian Bell,"1,6","1,8",https://www.allocine.fr/film/fichefilm_gen_cfi...,https://www.allocine.fr/personne/fichepersonne...,Lucia Aniello,U.S.A.,NA,Sony Pictures Releasing France,0,"$20,000,000",Anglais,97 842 entrées,2,3,1
2,#Alive,23 septembre 2021,1h 38min,Action,Drame,Epouvante-horreur,Hyung-cho Il,NA,Ah-In Yoo,Shin-Hye Park,Hyun-Wook Lee,NA,"3,0",https://www.allocine.fr/film/fichefilm_gen_cfi...,https://www.allocine.fr/personne/fichepersonne...,Hyung-cho Il,Corée du Sud,NA,Netflix,0,-,Coréen,NA,NA,0,NA
3,#Amour et déconnexion,31 mai 2021,1h 30min,Famille,NA,NA,Bradford May,NA,Mischa Barton,Sean Faris,B.J. Mitchell,NA,"3,0",https://www.allocine.fr/film/fichefilm_gen_cfi...,https://www.allocine.fr/personne/fichepersonne...,NA,U.S.A.,NA,NA,0,-,Anglais,NA,0,0,0
4,#Chef,29 octobre 2014,1h 54min,Comédie,Drame,NA,Jon Favreau,NA,Jon Favreau,John Leguizamo,Sofia Vergara,"2,8","3,6",https://www.allocine.fr/film/fichefilm_gen_cfi...,https://www.allocine.fr/personne/fichepersonne...,Jon Favreau,U.S.A.,NA,Sony Pictures Releasing France,1,-,Anglais,15 801 entrées,6,6,4
5,#FriendButMarried,14 novembre 2022,1h 42min,Biopic,Comédie dramatique,Romance,Rako Prijanto,NA,Adipati Dolken,Vanesha Prescilla,Refal Hady,NA,NA,https://www.allocine.fr/film/fichefilm_gen_cfi...,https://www.allocine.fr/personne/fichepersonne...,NA,Indonésie,NA,NA,0,-,Indonésien,NA,NA,0,0
6,#Home,18 août 2021,2h 38min,Drame,Famille,NA,Rojin Thomas,NA,Indrans,Sreenath Bhasi,Nalsen K. Gafoor,NA,NA,https://www.allocine.fr/film/fichefilm_gen_cfi...,https://www.allocine.fr/personne/fichepersonne...,Rojin Thomas,Inde,NA,NA,0,-,Malais,NA,NA,0,NA
7,#Horror,27 janvier 2019,1h 30min,Drame,Epouvante-horreur,Thriller,Tara Subkoff,NA,Mina Sundwall,Bridget McGarry,Haley Murphy,NA,"1,1",https://www.allocine.fr/film/fichefilm_gen_cfi...,https://www.allocine.fr/personne/fichepersonne...,Tara Subkoff,U.S.A.,NA,NA,0,-,Anglais,NA,0,0,0
8,#Jesuislà,5 février 2020,1h 38min,Comédie,Romance,NA,Eric Lartigau,NA,Alain Chabat,Doona Bae,Blanche Gardin,"3,0","2,4",https://www.allocine.fr/film/fichefilm_gen_cfi...,https://www.allocine.fr/personne/fichepersonne...,Eric Lartigau,France,Belgique,Gaumont Distribution,0,-,"Coréen, Français",193 851 entrées,13,2,0
9,#Moscou-Royan,21 novembre 2018,1h 23min,Comédie,NA,NA,Elena Cosson Kizilova,NA,Elena Cosson Kizilova,Donia Eden,Manuel Sorroche,NA,"3,1",https://www.allocine.fr/film/fichefilm_gen_cfi...,https://www.allocine.fr/personne/fichepersonne...,Elena Cosson Kizilova,France,NA,Leva Productions,0,-,Français,NA,NA,NA,0


In [54]:
coucou= pd.read_csv('movies_data.csv', encoding='utf-8-sig',sep=";")

In [55]:
coucou

,Titre,date_sortie,duree,genre1,genre2,genre3,réalisateur_1,réalisateur_2,actor_1,actor 2,actor 3,note_presse,note spectators,link,link realisator,Scenar,nationality 1,nationality 2,distributor,nominations,budget,language,box-office,real awards,first actor awards,Second actor awards
0,« Peu m'importe si l'Histoire nous considère c...,20 février 2019,2h 19min,Comédie,Drame,NaN,Radu Jude,NaN,Ioana Iacob,Alexandru Dabija,Alexandru Bogdan,"3,8","3,3",https://www.allocine.fr/film/fichefilm_gen_cfi...,https://www.allocine.fr/personne/fichepersonne...,Radu Jude,Roumanie,France,Météore Films,0,-,Roumain,901 entrées,6.0,0.0,0.0
1,# Pire soirée,2 août 2017,1h 41min,Comédie,NaN,NaN,Lucia Aniello,NaN,Scarlett Johansson,Kate McKinnon,Jillian Bell,"1,6","1,8",https://www.allocine.fr/film/fichefilm_gen_cfi...,https://www.allocine.fr/personne/fichepersonne...,Lucia Aniello,U.S.A.,NaN,Sony Pictures Releasing France,0,"$20,000,000",Anglais,97 842 entrées,2.0,3.0,1.0
2,#Alive,23 septembre 2021,1h 38min,Action,Drame,Epouvante-horreur,Hyung-cho Il,NaN,Ah-In Yoo,Shin-Hye Park,Hyun-Wook Lee,NaN,"3,0",https://www.allocine.fr/film/fichefilm_gen_cfi...,https://www.allocine.fr/personne/fichepersonne...,Hyung-cho Il,Corée du Sud,NaN,Netflix,0,-,Coréen,NaN,NaN,0.0,NaN
3,#Amour et déconnexion,31 mai 2021,1h 30min,Famille,NaN,NaN,Bradford May,NaN,Mischa Barton,Sean Faris,B.J. Mitchell,NaN,"3,0",https://www.allocine.fr/film/fichefilm_gen_cfi...,https://www.allocine.fr/personne/fichepersonne...,NaN,U.S.A.,NaN,NaN,0,-,Anglais,NaN,0.0,0.0,0.0
4,#Chef,29 octobre 2014,1h 54min,Comédie,Drame,NaN,Jon Favreau,NaN,Jon Favreau,John Leguizamo,Sofia Vergara,"2,8","3,6",https://www.allocine.fr/film/fichefilm_gen_cfi...,https://www.allocine.fr/personne/fichepersonne...,Jon Favreau,U.S.A.,NaN,Sony Pictures Releasing France,1,-,Anglais,15 801 entrées,6.0,6.0,4.0
5,#FriendButMarried,14 novembre 2022,1h 42min,Biopic,Comédie dramatique,Romance,Rako Prijanto,NaN,Adipati Dolken,Vanesha Prescilla,Refal Hady,NaN,NaN,https://www.allocine.fr/film/fichefilm_gen_cfi...,https://www.allocine.fr/personne/fichepersonne...,NaN,Indonésie,NaN,NaN,0,-,Indonésien,NaN,NaN,0.0,0.0
6,#Home,18 août 2021,2h 38min,Drame,Famille,NaN,Rojin Thomas,NaN,Indrans,Sreenath Bhasi,Nalsen K. Gafoor,NaN,NaN,https://www.allocine.fr/film/fichefilm_gen_cfi...,https://www.allocine.fr/personne/fichepersonne...,Rojin Thomas,Inde,NaN,NaN,0,-,Malais,NaN,NaN,0.0,NaN
7,#Horror,27 janvier 2019,1h 30min,Drame,Epouvante-horreur,Thriller,Tara Subkoff,NaN,Mina Sundwall,Bridget McGarry,Haley Murphy,NaN,"1,1",https://www.allocine.fr/film/fichefilm_gen_cfi...,https://www.allocine.fr/personne/fichepersonne...,Tara Subkoff,U.S.A.,NaN,NaN,0,-,Anglais,NaN,0.0,0.0,0.0
8,#Jesuislà,5 février 2020,1h 38min,Comédie,Romance,NaN,Eric Lartigau,NaN,Alain Chabat,Doona Bae,Blanche Gardin,"3,0","2,4",https://www.allocine.fr/film/fichefilm_gen_cfi...,https://www.allocine.fr/personne/fichepersonne...,Eric Lartigau,France,Belgique,Gaumont Distribution,0,-,"Coréen, Français",193 851 entrées,13.0,2.0,0.0
9,#Moscou-Royan,21 novembre 2018,1h 23min,Comédie,NaN,NaN,Elena Cosson Kizilova,NaN,Elena Cosson Kizilova,Donia Eden,Manuel Sorroche,NaN,"3,1",https://www.allocine.fr/film/fichefilm_gen_cfi...,https://www.allocine.fr/personne/fichepersonne...,Elena Cosson Kizilova,France,NaN,Leva Productions,0,-,Français,NaN,NaN,NaN,0.0
